<h2> <b>Set up </b></h2>

In [ ]:
# laod data from Google drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
# check the GPU info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
# import packages
import tensorflow as tf
print(tf.__version__)

import keras
from keras.layers import Input, Dense, Conv2D, Reshape, BatchNormalization, LeakyReLU, Conv2DTranspose
from keras.layers import Flatten, Reshape, LSTM
from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.losses import binary_crossentropy
from keras import backend as K
from keras.optimizers import RMSprop, Adam

import numpy as np
import matplotlib.pyplot as plt 
import cv2
import skimage
import imutils
%matplotlib inline

np.random.seed(42)

from imutils import face_utils
import numpy as np
import argparse
import imutils
import dlib
import cv2

<h1> <b>Load data from video (.mp4 -> .npy)</b></h1>

In [ ]:
# function for save the frames in the .mp4 file into .npy file

def load_data_from_video(video_file, colored=False, resize_factor=10, save_npy=True):

    ''' --------------- parameter explanation ---------------
    video_file: 
      video file name
    colored: 
      False if the frames in the .npy file are expected to be in grayscale; Ture if the frames in the .npy file are expected to be colored
    resize_factor: 
      the width and height of the frames in the .npy file is how many times smaller than the width and height of the frames in the .mp4 file
    ---------------------------------------------------------''' 

    cap = cv2.VideoCapture(video_file + ".mp4")
    num_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

    print("the total number of frames is {}".format(num_frames))

    frame_count = 1

    ret, frame0 = cap.read()

    # convert the picture to gray or BGR
    if colored:
        frame0 = cv2.cvtColor(frame0, cv2.COLOR_BGR2RGB)
    else:
        frame0 = cv2.cvtColor(frame0, cv2.COLOR_BGR2GRAY)

    # resize the picture for lighter training
    if resize_factor != 1:
        frame0 = imutils.resize(frame0, width=int(1280/resize_factor))

    # reshape the picture
    if colored:
        frame0 = np.reshape(frame0, (1, frame0.shape[0], frame0.shape[1], 3))
    else:
        frame0 = np.reshape(frame0, (1, frame0.shape[0], frame0.shape[1], 1))

    data = frame0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if colored:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        else:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # resize the picture for lighter training
        if resize_factor != 1:
            frame = imutils.resize(frame, width=int(1280/resize_factor))

        if colored:
            frame = np.reshape(frame, (1, frame.shape[0], frame.shape[1], 3))
        else:
            frame = np.reshape(frame, (1, frame.shape[0], frame.shape[1], 1))

        data = np.concatenate((data, frame), axis=0)

        frame_count += 1
        if frame_count % 100 == 0:
            print("the current loading frame is {}".format(frame_count))

    cap.release()

    print(data.shape)

    if save_npy:
        if colored:
            np.save(video_file + "_colored_data.npy", data)
        else:
            np.save(video_file + "gray_data.npy", data)


In [ ]:
video_file = "human_aging"
colored = False

# uncomment if the video is not loaded before
# load_data_from_video(video_file, colored=colored, resize_factor=2, save_npy=True)


In [ ]:
# load the .npy file from Google drive
data = np.load("/content/drive/MyDrive/human_aging_gray_data_large.npy")
    
data = np.reshape(data, (7428, 360, 640))

frame_num = data.shape[0]

In [ ]:
#display some examples

plt.figure(figsize=(10,6))

plt.subplot(121)
plt.imshow(data[2000], cmap=plt.cm.gray)
plt.axis('off')

plt.subplot(122)
plt.imshow(data[7000], cmap=plt.cm.gray)
plt.axis('off')

<h1> <b>Crop the frames </b></h1>

In [ ]:
''' --------------- cropping method 1 ---------------
    Goal: crop the image to reduce computational cost while
      1. keep the entire face as much as possible (including hair)
      2. keep the information in height changes and make the growing up trend visible
    Parameter explanation:
      1. the resize_factor is the same resize_factor to what in the load data from video function above
      2. center_x & center_y: center_x and center_y are gained by input the image into the get_center function below
    ---------------------------------------------------------''' 

def crop(image, resize_factor, center_x, center_y):
    h = int(70 * 10/resize_factor)
    w = int(55 * 10/resize_factor)
    x = int(center_x - w/2)
    y = 0
    #box = (x, y, w, h)
    roi = image[y:y+h, x:x+w]
    #roi = image.crop(box)
    return roi

In [ ]:
''' --------------- cropping method 2 ---------------
    Goal: crop the image to reduce computational cost while
      1. make the position of the center of the face to a fixed point in each frame
    Parameter explanation:
      1. resize_factor: the resize_factor is the same resize_factor to what in the load data from video function above
      2. center_x & center_y: center_x and center_y are gained by input the image into the get_center function below
    ---------------------------------------------------------''' 

def crop(image, resize_factor, center_x, center_y):
  w = int(55 * 10/resize_factor)
  roi = image[int(center_y-34 * 10/resize_factor):int(center_y+22 * 10/resize_factor), int(center_x-w/2):int(center_x+w/2)]
  return roi

In [ ]:
# calculate the center of the face with facial landmarks
# dlib open source with: shape_predictor_68_face_landmarks.dat


def get_center(image):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("/content/drive/MyDrive/shape_predictor_68_face_landmarks.dat")
    #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = image
    rects = detector(gray, 1)
    X = []
    Y = []
    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        for (name, (i, j)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
            for (x, y) in shape[i:j]:
#                 cv2.circle(image, (x, y), 1, (255, 0, 0), -1)
#                 cv2.circle(gray, (x, y), 2, (0, 0, 255), -1)
            #print("x: " + str(x) + ", y: " + str(y))
                X.append(x)
                Y.append(y)
        #cv2.circle(image, (int(np.mean(X)), int(np.mean(Y))), 1, (0, 255, 0), -1)
        center_x = int(np.mean(X))
        center_y = int(np.mean(Y))
        #cv2.circle(image, center_x, center_y, 1, (0, 255, 0), -1)
        #output = face_utils.visualize_facial_landmarks(image, shape)
        #image = crop1(image, 10, center_x, center_y)
        # print("center_y: " + str(center_y))
        # print("max(Y): " + str(max(Y)))
        # print("max(Y)-center_y: " + str(max(Y)-center_y))
        range_X = int(np.max(X))-int(np.min(X))
        #print("range X: " + str(range_X) + "\n")
    #plt.figure(figsize=(10,6))
    #plt.imshow(image)
    return center_x, center_y

In [ ]:
# plot some sample frames after cropping to test if the parameters in the cropping methods are correct  

plt.figure(figsize=(20,10))
a, b =2, 4
for i in range(8):  
    plt.subplot(a, b, i+1)
    image = data[1000*i]
    x, y = get_center(image) 
    roi = crop(image, 2, x, y) # works for either cropping method 1 or cropping method 2
    plt.imshow(roi, cmap=plt.cm.gray)
    plt.axis('off')
    plt.title('frame: ' + str(1000*i))
plt.show()

In [ ]:
# crop the frames to reduce the computational cost and save them as .npy file to Google drive
crop_data = []
for i in range(data.shape[0]):
  image = data[i]
  x, y = get_center(image)
  roi = crop(image, 2, x, y)
  crop_data.append(roi)

#np.save("/content/drive/MyDrive/cropped_data_large_fixed.npy",crop_data)